In [1]:
import pandas as pd 
import numpy as np 
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, MinMaxScaler
from sklearn.pipeline import Pipeline

In [2]:
df = pd.read_csv('./train.csv')

In [3]:
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [23]:
numerical_col = ['LotFrontage', 'MasVnrArea', 'BsmtFinSF1']
one_hot_encoding_cols = [
    'MSZoning', 'Alley', 'LandSlope', 'Neighborhood', 'BldgType', 'HouseStyle',
    'RoofStyle', 'RoofMatl', 'Exterior1st', 'MasVnrType', 'Exterior2nd', 'Foundation',
    'Heating', 'CentralAir', 'GarageType', 'Fence', 'MiscFeature', 'Electrical'
]
ordinal_encoding_cols = [
    'Street', 'LotShape', 'LandContour', 'LotConfig', 'Utilities', 'Condition1', 
    'Condition2', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
    'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 
    'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC'
]
pt_list =[]

In [34]:
def separate_columns_by_dtype(df):
    int_columns = df.select_dtypes(include=['int64']).columns.tolist()
    float_columns = df.select_dtypes(include=['float64']).columns.tolist()
    object_columns = df.select_dtypes(include=['object']).columns.tolist()

    return int_columns, float_columns, object_columns

In [35]:
int_columns, float_columns, object_columns = separate_columns_by_dtype(df)

In [25]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

In [26]:
one = OneHotEncoder(sparse_output=False)
odi = OrdinalEncoder()

In [27]:
from sklearn.compose import ColumnTransformer

In [28]:
# processing pipeline for numerical columns
numeric_processor = Pipeline(
    steps = [('imputation_mean', SimpleImputer(missing_values= np.nan, strategy='mean')),
            ('min_max_scaler', MinMaxScaler())
            ]
)

In [29]:
numeric_processor

Pipeline(steps=[('imputation_mean', SimpleImputer()),
                ('min_max_scaler', MinMaxScaler())])

In [30]:
# preprocessin for categorical columns
odinal_cat = Pipeline(
    steps = [
            ('imputation_constant', SimpleImputer(fill_value= 'None', strategy='constant')), 
            ('odinal', OrdinalEncoder())
    ]
)

In [31]:
onehot_cat = Pipeline(
    steps = [
            ('imputation_constant', SimpleImputer(fill_value= 'None', strategy='constant')), 
            ('oneHot', OneHotEncoder()),
    ]
)

In [36]:
preprocessor = ColumnTransformer(
                    [('numerical', numeric_processor, int_columns),
                    ('odinal_cat', odinal_cat, ordinal_encoding_cols),
                    ('onehot_cat', onehot_cat, one_hot_encoding_cols),])

In [37]:
preprocessor

ColumnTransformer(transformers=[('numerical',
                                 Pipeline(steps=[('imputation_mean',
                                                  SimpleImputer()),
                                                 ('min_max_scaler',
                                                  MinMaxScaler())]),
                                 ['Id', 'MSSubClass', 'LotArea', 'OverallQual',
                                  'OverallCond', 'YearBuilt', 'YearRemodAdd',
                                  'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
                                  'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
                                  'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
                                  'BsmtHalfBath', 'F...
                                ('onehot_cat',
                                 Pipeline(steps=[('imputation_constant',
                                                  SimpleImputer(fill_value='None',
                                                                strategy='constant')),
                                                 ('oneHot', OneHotEncoder())]),
                                 ['MSZoning', 'Alley', 'LandSlope',
                                  'Neighborhood', 'BldgType', 'HouseStyle',
                                  'RoofStyle', 'RoofMatl', 'Exterior1st',
                                  'MasVnrType', 'Exterior2nd', 'Foundation',
                                  'Heating', 'CentralAir', 'GarageType',
                                  'Fence', 'MiscFeature', 'Electrical'])])

In [38]:
# pandas_df = preprocessor.fit_transform(df)

In [40]:
from sklearn.pipeline import make_pipeline

In [43]:
final_pipeling = make_pipeline(
    preprocessor
)

In [44]:
final_pipeling

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputation_mean',
                                                                   SimpleImputer()),
                                                                  ('min_max_scaler',
                                                                   MinMaxScaler())]),
                                                  ['Id', 'MSSubClass',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'BsmtFinSF1',
                                                   'BsmtFinSF2', 'BsmtUnfSF',
                                                   'TotalBsmtSF', '1stFlrSF',
                                                   '2ndFlrSF', 'LowQualFinSF',
                                                   'GrLivA...
                                                 ('onehot_cat',
                                                  Pipeline(steps=[('imputation_constant',
                                                                   SimpleImputer(fill_value='None',
                                                                                 strategy='constant')),
                                                                  ('oneHot',
                                                                   OneHotEncoder())]),
                                                  ['MSZoning', 'Alley',
                                                   'LandSlope', 'Neighborhood',
                                                   'BldgType', 'HouseStyle',
                                                   'RoofStyle', 'RoofMatl',
                                                   'Exterior1st', 'MasVnrType',
                                                   'Exterior2nd', 'Foundation',
                                                   'Heating', 'CentralAir',
                                                   'GarageType', 'Fence',
                                                   'MiscFeature',
                                                   'Electrical'])]))])